# Fraud Detection in Electricity and Gas Consumption Challenge

The Tunisian Company of Electricity and Gas (STEG) is a public and a non-administrative company, it is responsible for delivering electricity and gas across Tunisia. The company suffered tremendous losses in the order of 200 million Tunisian Dinars due to fraudulent manipulations of meters by consumers.

Using the client’s billing history, the aim of the challenge is to detect and recognize clients involved in fraudulent activities.

The solution will enhance the company’s revenues and reduce the losses caused by such fraudulent activities.

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Import datasets

invoice_train = pd.read_csv("train/invoice_train.csv", low_memory=False)
invoice_test = pd.read_csv("test/invoice_test.csv", low_memory=False)
client_train = pd.read_csv("train/client_train.csv", low_memory=False)
client_test = pd.read_csv("test/client_test.csv", low_memory=False)
Sample_Submission = pd.read_csv("SampleSubmission.csv", low_memory=False)

In [3]:
invoice_train

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,ELEC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4476744,train_Client_99998,2005-08-19,10,1253571,0,202,9,1,400,135,0,0,3197,3732,8,ELEC
4476745,train_Client_99998,2005-12-19,10,1253571,0,202,6,1,200,6,0,0,3732,3938,4,ELEC
4476746,train_Client_99999,1996-09-25,11,560948,0,203,6,1,259,0,0,0,13884,14143,4,ELEC
4476747,train_Client_99999,1996-05-28,11,560948,0,203,6,1,603,0,0,0,13281,13884,4,ELEC


In [4]:
print(invoice_train.shape, invoice_test.shape, client_train.shape, client_test.shape)

(4476749, 16) (1939730, 16) (135493, 6) (58069, 5)


The Target is located in the client dataset

In [5]:
client_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   disrict        135493 non-null  int64  
 1   client_id      135493 non-null  object 
 2   client_catg    135493 non-null  int64  
 3   region         135493 non-null  int64  
 4   creation_date  135493 non-null  object 
 5   target         135493 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 6.2+ MB


In [6]:
invoice_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4476749 entries, 0 to 4476748
Data columns (total 16 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   client_id             object
 1   invoice_date          object
 2   tarif_type            int64 
 3   counter_number        int64 
 4   counter_statue        object
 5   counter_code          int64 
 6   reading_remarque      int64 
 7   counter_coefficient   int64 
 8   consommation_level_1  int64 
 9   consommation_level_2  int64 
 10  consommation_level_3  int64 
 11  consommation_level_4  int64 
 12  old_index             int64 
 13  new_index             int64 
 14  months_number         int64 
 15  counter_type          object
dtypes: int64(12), object(4)
memory usage: 546.5+ MB


In [7]:
invoice_train.isnull().sum()

client_id               0
invoice_date            0
tarif_type              0
counter_number          0
counter_statue          0
counter_code            0
reading_remarque        0
counter_coefficient     0
consommation_level_1    0
consommation_level_2    0
consommation_level_3    0
consommation_level_4    0
old_index               0
new_index               0
months_number           0
counter_type            0
dtype: int64

In [8]:
client_train.isnull().sum()

disrict          0
client_id        0
client_catg      0
region           0
creation_date    0
target           0
dtype: int64

No missing value in dataset

In [9]:
for i in invoice_train.columns:
    print(f"{i} - {invoice_train[i].nunique()}")

client_id - 135493
invoice_date - 8275
tarif_type - 17
counter_number - 201893
counter_statue - 12
counter_code - 42
reading_remarque - 8
counter_coefficient - 16
consommation_level_1 - 8295
consommation_level_2 - 12576
consommation_level_3 - 2253
consommation_level_4 - 12075
old_index - 155648
new_index - 157980
months_number - 1370
counter_type - 2


In [10]:
for i in client_train.columns:
    print(f"{i} - {client_train[i].nunique()}")

disrict - 4
client_id - 135493
client_catg - 3
region - 25
creation_date - 8088
target - 2


In [11]:
invoice_train.dtypes

client_id               object
invoice_date            object
tarif_type               int64
counter_number           int64
counter_statue          object
counter_code             int64
reading_remarque         int64
counter_coefficient      int64
consommation_level_1     int64
consommation_level_2     int64
consommation_level_3     int64
consommation_level_4     int64
old_index                int64
new_index                int64
months_number            int64
counter_type            object
dtype: object

In [12]:
client_train.dtypes

disrict            int64
client_id         object
client_catg        int64
region             int64
creation_date     object
target           float64
dtype: object

In [13]:
invoice_train.describe()

,tarif_type,counter_number,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number
count,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06
mean,2.012804e+01,1.230587e+11,1.724884e+02,7.321702e+00,1.003040e+00,4.109795e+02,1.093225e+02,2.030620e+01,5.292588e+01,1.776700e+04,1.834970e+04,4.483095e+01
std,1.347256e+01,1.657267e+12,1.338871e+02,1.571654e+00,3.083466e-01,7.573080e+02,1.220123e+03,1.574239e+02,8.754725e+02,4.036693e+04,4.095321e+04,3.128335e+03
min,8.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.100000e+01,1.211080e+05,5.000000e+00,6.000000e+00,1.000000e+00,7.900000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.791000e+03,2.056000e+03,4.000000e+00
50%,1.100000e+01,4.945610e+05,2.030000e+02,8.000000e+00,1.000000e+00,2.740000e+02,0.000000e+00,0.000000e+00,0.000000e+00,7.690000e+03,8.192000e+03,4.000000e+00
75%,4.000000e+01,1.115161e+06,2.070000e+02,9.000000e+00,1.000000e+00,6.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,2.166000e+04,2.234300e+04,4.000000e+00
max,4.500000e+01,2.798115e+13,6.000000e+02,4.130000e+02,5.000000e+01,9.999100e+05,9.990730e+05,6.449200e+04,5.479460e+05,2.800280e+06,2.870972e+06,6.366240e+05


In [14]:
client_train.describe()

,disrict,client_catg,region,target
count,135493.000000,135493.000000,135493.000000,135493.000000
mean,63.511222,11.512506,206.159809,0.055841
std,3.354400,4.423761,104.207044,0.229614
min,60.000000,11.000000,101.000000,0.000000
25%,62.000000,11.000000,103.000000,0.000000
50%,62.000000,11.000000,107.000000,0.000000
75%,69.000000,11.000000,307.000000,0.000000
max,69.000000,51.000000,399.000000,1.000000


# Feature Engineering

In [15]:
#convert the column invoice_date to date time format on both the invoice train and invoice test
for df in [invoice_train,invoice_test]:
    df['invoice_date'] = pd.to_datetime(df['invoice_date'])

In [16]:
#encode labels in categorical column
invoice_train['counter_type']=invoice_train['counter_type'].map({"ELEC":0,"GAZ":1})
invoice_test['counter_type']=invoice_test['counter_type'].map({"ELEC":0,"GAZ":1})

In [17]:
#convert categorical columns to int for model
client_train['client_catg'] = client_train['client_catg'].astype(int)
client_train['disrict'] = client_train['disrict'].astype(int)

client_test['client_catg'] = client_test['client_catg'].astype(int)
client_test['disrict'] = client_test['disrict'].astype(int)

In [18]:
def aggregate_by_client_id(invoice_data):
    aggs = {}
    aggs['consommation_level_1'] = ['mean']
    aggs['consommation_level_2'] = ['mean']
    aggs['consommation_level_3'] = ['mean']
    aggs['consommation_level_4'] = ['mean']

    agg_trans = invoice_data.groupby(['client_id']).agg(aggs)
    agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = (invoice_data.groupby('client_id')
            .size()
            .reset_index(name='{}transactions_count'.format('1')))
    return pd.merge(df, agg_trans, on='client_id', how='left')

In [19]:
#group invoice data by client_id
agg_train = aggregate_by_client_id(invoice_train)

In [20]:
print(agg_train.shape)
agg_train.head()

(135493, 6)


,client_id,1transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean
0,train_Client_0,35,352.400000,10.571429,0.000000,0.000000
1,train_Client_1,37,557.540541,0.000000,0.000000,0.000000
2,train_Client_10,18,798.611111,37.888889,0.000000,0.000000
3,train_Client_100,20,1.200000,0.000000,0.000000,0.000000
4,train_Client_1000,14,663.714286,104.857143,117.357143,36.714286


In [21]:
#merge aggregate data with client dataset
train = pd.merge(client_train,agg_train, on='client_id', how='left')

In [22]:
#aggregate test set
agg_test = aggregate_by_client_id(invoice_test)
test = pd.merge(client_test,agg_test, on='client_id', how='left')

In [23]:
train.shape, test.shape

((135493, 11), (58069, 10))

In [24]:
#drop redundant columns
sub_client_id = test['client_id']
drop_columns = ['client_id', 'creation_date']

for col in drop_columns:
    if col in train.columns:
        train.drop([col], axis=1, inplace=True)
    if col in test.columns:
        test.drop([col], axis=1, inplace=True)

In [25]:
train

,disrict,client_catg,region,target,1transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean
0,60,11,101,0.0,35,352.400000,10.571429,0.000000,0.000000
1,69,11,107,0.0,37,557.540541,0.000000,0.000000,0.000000
2,62,11,301,0.0,18,798.611111,37.888889,0.000000,0.000000
3,69,11,105,0.0,20,1.200000,0.000000,0.000000,0.000000
4,62,11,303,0.0,14,663.714286,104.857143,117.357143,36.714286
...,...,...,...,...,...,...,...,...,...
135488,62,11,304,0.0,71,1.957746,0.000000,0.000000,0.000000
135489,63,11,311,0.0,41,185.853659,0.756098,0.000000,0.000000
135490,63,11,311,0.0,36,273.083333,0.000000,0.000000,0.000000
135491,60,11,101,0.0,2,300.000000,70.500000,0.000000,0.000000


In [26]:
y_train_data = train['target']
X_train_data = train[['disrict','client_catg','region','1transactions_count','consommation_level_1_mean',
                      'consommation_level_2_mean','consommation_level_3_mean','consommation_level_4_mean']]

In [27]:
X_train_data

,disrict,client_catg,region,1transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean
0,60,11,101,35,352.400000,10.571429,0.000000,0.000000
1,69,11,107,37,557.540541,0.000000,0.000000,0.000000
2,62,11,301,18,798.611111,37.888889,0.000000,0.000000
3,69,11,105,20,1.200000,0.000000,0.000000,0.000000
4,62,11,303,14,663.714286,104.857143,117.357143,36.714286
...,...,...,...,...,...,...,...,...
135488,62,11,304,71,1.957746,0.000000,0.000000,0.000000
135489,63,11,311,41,185.853659,0.756098,0.000000,0.000000
135490,63,11,311,36,273.083333,0.000000,0.000000,0.000000
135491,60,11,101,2,300.000000,70.500000,0.000000,0.000000


In [28]:
y_train_data

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
135488    0.0
135489    0.0
135490    0.0
135491    0.0
135492    0.0
Name: target, Length: 135493, dtype: float64

# Building Model

In [29]:
# Import various models
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [30]:
# Split the dataset into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X_train_data, y_train_data, test_size=0.3, random_state=8)

In [37]:
# Model 1: Logistic regression
log_model = LogisticRegression()

log_model.fit(X_train, y_train)

LogisticRegression()

In [38]:
y_pred_log = log_model.predict(X_val)

# Calculate the AUC - ROC score
roc_auc = roc_auc_score(y_val, y_pred_log)
print(f"AUC-ROC for logistic regression model: {roc_auc}")

AUC-ROC for logistic regression model: 0.5013853890322364


In [40]:
# model 2: KNN model using a grid search
pipe = Pipeline([
    ("scale", StandardScaler()),
    ("model", KNeighborsClassifier()),
])

knn_model = GridSearchCV(estimator=pipe, 
                         param_grid={'model__n_neighbors' : [1, 2, 3 , 4, 5, 6, 7, 8, 9, 10]},
                         cv=3)

In [41]:
knn_model.fit(X_train, y_train)

AttributeError: 'GridSearchCV' object has no attribute 'best_params'

In [42]:
pd.DataFrame(knn_model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.775221,0.291617,1.960035,0.149151,1,{'model__n_neighbors': 1},0.897549,0.900870,0.901945,0.900121,0.001871,10
1,0.768001,0.250013,2.494615,0.109697,2,{'model__n_neighbors': 2},0.939333,0.939807,0.939997,0.939712,0.000279,8
2,0.810679,0.349498,2.448650,0.174924,3,{'model__n_neighbors': 3},0.932406,0.933418,0.933291,0.933038,0.000450,9
3,0.725314,0.241300,2.605344,0.187810,4,{'model__n_neighbors': 4},0.942084,0.942432,0.942053,0.942190,0.000172,6
4,0.755033,0.227196,2.821980,0.108570,5,{'model__n_neighbors': 5},0.939649,0.940440,0.940313,0.940134,0.000347,7
5,0.741342,0.253878,2.759020,0.049822,6,{'model__n_neighbors': 6},0.943002,0.943033,0.942844,0.942960,0.000083,3
6,0.743687,0.255521,2.884642,0.080525,7,{'model__n_neighbors': 7},0.942306,0.942306,0.942179,0.942264,0.000060,5
7,0.759981,0.284974,3.001678,0.063140,8,{'model__n_neighbors': 8},0.943381,0.943065,0.943255,0.943234,0.000130,2
8,0.775663,0.246295,3.183995,0.018990,9,{'model__n_neighbors': 9},0.942812,0.942938,0.942780,0.942844,0.000068,4
9,0.779700,0.237458,3.160662,0.081003,10,{'model__n_neighbors': 10},0.943381,0.943192,0.943445,0.943339,0.000108,1


In [48]:
best_knn_model = KNeighborsClassifier(n_neighbors=3)

In [50]:
best_knn_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [51]:
y_pred_knn = best_knn_model.predict(X_val)

# Calculate the AUC - ROC score
roc_auc_knn = roc_auc_score(y_val, y_pred_knn)
print(f"AUC-ROC for KNN regression model: {roc_auc_knn}")

AUC-ROC for KNN regression model: 0.5112114869067638


In [53]:
pipe2 = Pipeline([
    ("scale", StandardScaler()),
    ("model", XGBClassifier())
])

In [54]:
xgb_mod = GridSearchCV(estimator=pipe2,
                       param_grid={
                           'model__min_child_weighth': [1, 5, 10],
                           'model__gamma': [0.5, 1],
                           'model__subsample': [0.6, 0.8, 1.0],
                           'model__max_depth': [3, 5]
                       },
                       n_jobs=3,
                       verbose=3,
                       cv=5)

In [56]:
xgb_mod.fit(X_train, y_train)
pd.DataFrame(xgb_mod.cv_results_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


C:\Users\Samuel Ladipo\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning: [10:37:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "min_child_weighth" } are not used.

  warnings.warn(smsg, UserWarning)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__gamma,param_model__max_depth,param_model__min_child_weighth,param_model__subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.316290,0.089012,0.043400,0.006469,0.5,3,1,0.6,"{'model__gamma': 0.5, 'model__max_depth': 3, '...",0.943698,0.943592,0.943645,0.943645,0.943329,0.943582,0.000131,19
1,1.176800,0.021684,0.040999,0.007643,0.5,3,1,0.8,"{'model__gamma': 0.5, 'model__max_depth': 3, '...",0.943645,0.943698,0.943329,0.943750,0.943645,0.943613,0.000148,13
2,0.693398,0.027753,0.037201,0.008010,0.5,3,1,1.0,"{'model__gamma': 0.5, 'model__max_depth': 3, '...",0.943803,0.943645,0.943645,0.943645,0.943803,0.943708,0.000077,7
3,1.243801,0.033008,0.034998,0.002608,0.5,3,5,0.6,"{'model__gamma': 0.5, 'model__max_depth': 3, '...",0.943698,0.943592,0.943645,0.943645,0.943329,0.943582,0.000131,19
4,1.169398,0.021748,0.034200,0.006735,0.5,3,5,0.8,"{'model__gamma': 0.5, 'model__max_depth': 3, '...",0.943645,0.943698,0.943329,0.943750,0.943645,0.943613,0.000148,13
5,0.703599,0.043070,0.042202,0.004167,0.5,3,5,1.0,"{'model__gamma': 0.5, 'model__max_depth': 3, '...",0.943803,0.943645,0.943645,0.943645,0.943803,0.943708,0.000077,7
6,1.256398,0.024450,0.038399,0.007144,0.5,3,10,0.6,"{'model__gamma': 0.5, 'model__max_depth': 3, '...",0.943698,0.943592,0.943645,0.943645,0.943329,0.943582,0.000131,19
7,1.219995,0.081382,0.042802,0.006045,0.5,3,10,0.8,"{'model__gamma': 0.5, 'model__max_depth': 3, '...",0.943645,0.943698,0.943329,0.943750,0.943645,0.943613,0.000148,13
8,1.123199,0.310725,0.083999,0.075463,0.5,3,10,1.0,"{'model__gamma': 0.5, 'model__max_depth': 3, '...",0.943803,0.943645,0.943645,0.943645,0.943803,0.943708,0.000077,7
9,1.897798,0.156338,0.058198,0.004996,0.5,5,1,0.6,"{'model__gamma': 0.5, 'model__max_depth': 5, '...",0.942801,0.942749,0.942960,0.943487,0.943012,0.943002,0.000261,31


In [58]:
print(xgb_mod.best_params_)

{'model__gamma': 1, 'model__max_depth': 5, 'model__min_child_weighth': 1, 'model__subsample': 1.0}


In [59]:
# fit by setting best parameters and Evaluate model
best_xgb_model = XGBClassifier(min_child_weight=1, gamma=1, subsample=1.0, max_depth=5)

best_xgb_model.fit(X_train, y_train)
y_pred_xgb = best_xgb_model.predict(X_val)

# Get AUC_ROC score
roc_auc_xgb = roc_auc_score(y_val, y_pred_xgb)
print(f"AUC-ROC for KNN xgb model: {roc_auc_xgb}")

AUC-ROC for KNN xgb model: 0.5007512699612915


Need to go back to my feature engineering. The results for the prediction are not so good